# Task - 5
# Customer Retention Strategies

In [3]:
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestClassifier

In [4]:
df=pd.read_csv("Telco-Customer-Churn.csv")

In [5]:
df=df.drop(columns=['CustomerID'],errors='ignore')

In [6]:
df['Churn']=df['Churn'].map({'Yes':1,'No':0})

## Identify categorical columns

In [7]:
categorical_columns = df.select_dtypes(include=['object']).columns

## OneHotEncode categorical columns

In [10]:
from sklearn.preprocessing import OneHotEncoder
encoder = OneHotEncoder(sparse_output=False, drop='first')
encoded = pd.DataFrame(encoder.fit_transform(df[categorical_columns]), 
                       columns=encoder.get_feature_names_out(categorical_columns))

## Combine encoded columns with numeric columns

In [11]:
df = pd.concat([df.select_dtypes(exclude=['object']), encoded], axis=1)

## Fill missing values with median

In [12]:
df.fillna(df.median(), inplace=True)

## Seprate features and taget variable

In [15]:
X = df.drop(columns=['Churn'])
y = df['Churn']

## Identify key Factors using Feature Importance
### Train a Random Forest Classifier

In [16]:
model = RandomForestClassifier(random_state=42)
model.fit(X, y)

RandomForestClassifier(random_state=42)

## Extract feature importance

In [17]:
feature_importance = pd.DataFrame({'Feature': X.columns, 'Importance': model.feature_importances_})
feature_importance = feature_importance.sort_values(by='Importance', ascending=False)

In [18]:
print("Key Factors Influencing Churn:")
print(feature_importance.head())

Key Factors Influencing Churn:
                             Feature  Importance
1                             tenure    0.084299
2                     MonthlyCharges    0.054242
7051     InternetService_Fiber optic    0.025475
7069  PaymentMethod_Electronic check    0.024179
7066               Contract_Two year    0.021114


## Propose Retention Strategies

In [19]:
print("\nRetention Strategies:")
print("1. Focus on improving services for customers with short tenure.")
print("2. Offer discounts or incentives for customers on month-to-month contracts.")
print("3. Address complaints from customers with high monthly charges.")


Retention Strategies:
1. Focus on improving services for customers with short tenure.
2. Offer discounts or incentives for customers on month-to-month contracts.
3. Address complaints from customers with high monthly charges.


## Calculate Customer Lifetime Value(CLV)

In [20]:
df['CLV'] = df['MonthlyCharges'] * df['tenure']
high_value_customers = df[df['CLV'] > df['CLV'].quantile(0.9)]
at_risk_customers = high_value_customers[high_value_customers['Churn'] == 1]

In [21]:
print("\nHigh-Value Customers at Risk of Churning:")
print(at_risk_customers[['CLV', 'tenure', 'MonthlyCharges']])


High-Value Customers at Risk of Churning:
          CLV  tenure  MonthlyCharges
104   7480.00      68          110.00
339   6514.20      63          103.40
402   6862.05      69           99.45
445   6354.00      60          105.90
609   6841.25      65          105.25
...       ...     ...             ...
6785  6807.60      61          111.60
6853  5999.40      54          111.10
6945  6881.00      70           98.30
6972  6269.20      56          111.95
7034  6897.65      67          102.95

[86 rows x 3 columns]
